In [ ]:
from find_source import full_html_and_txt

In [ ]:
full_html_and_txt('../data/250611_03:56:34/*.fits', '../data/250611_03:56:34/polaris.json')

In [ ]:
def full_html_and_txt(folder: str, threshold: float = 0.01, radius_buffer: float = 5.0, ext_threshold: float = 0.001):
    '''
    From a folder of FITS files, creates source_info.html with observation information table, source figures, and source information table
    and creates interesting_field.txt with names of objects with any (possibly) interesting detections.

    Parameters
    ----------
    folder : str
        The path of the folder containing the FITS files to be analyzed.
    threshold : float (optional)
        The threshold for a significant detection.
        If the probability of detecting the center region's maximum flux assuming no source in the image
        is less than this threshold, then the detection is deemed significant.
        If no value is given, defaults to 0.01.
    radius_buffer : float (optional)
        The amount of buffer, in arcsec, to add to the beam FWHM to get the initial search radius.
        If no value is given, defaults to 5 arcsec.
    ext_threshold : float (optional)
        The probability that an external peak must be below for it to be considered an external source.
        If no value is given, defaults to 0.001.
    '''

    start_html()

    if folder[-1] != '/':
        folder += '/'

    json_file = f'{folder}polaris.json'

    obs_info_to_html(json_file)

    final_catalog = {}
    file = open(json_file, 'r')
    obs_dict = json.load(file)
    sci_targs = obs_dict['sciTargs']

    txt = open('../html/interesting_fields.txt', 'w')

    for file in glob.glob(f'{folder}*.fits'):
        try:
            fig_to_html(file, radius_buffer=radius_buffer, ext_threshold=ext_threshold)
            obj = fits.getheader(file)['OBJECT']
            if obj in sci_targs:
                catalog = make_catalog(file, threshold=threshold, radius_buffer=radius_buffer, ext_threshold=ext_threshold)

                #add field name to .txt file if it is a science target with a significant detection in the initial inclusion region
                if catalog != None:
                    for key, value in catalog.items():
                        if value['internal'] == True:
                            txt.write(f'{obj}\n')
                final_catalog = combine_catalogs(final_catalog, catalog)
        except:
            print(f'Try again for {file}')

    txt.close()

    catalog_to_html(final_catalog)
    end_html()

    plt.close('all')